In [3]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

True

In [8]:
spotify_id = os.getenv('SPOTIFY_ID')
spotify_key = os.getenv('SPOTIFY_TOKEN')

In [9]:
import requests
from base64 import b64encode

def get_spotify_token(client_id, client_secret):
    # Encode as Base64
    credentials = b64encode(f"{client_id}:{client_secret}".encode()).decode('utf-8')

    headers = {
        "Authorization": f"Basic {credentials}"
    }

    body = {
        "grant_type": "client_credentials"
    }

    token_url = "https://accounts.spotify.com/api/token"
    response = requests.post(token_url, headers=headers, data=body)
    response_data = response.json()
    return response_data['access_token']

access_token = get_spotify_token(spotify_id, spotify_key)

In [15]:
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
id="11dFghVXANMlKmJXsNCbNl"
endpoint = f"https://api.spotify.com/v1/audio-features/{id}"

In [16]:
response = requests.get(endpoint, headers=headers)


In [17]:
response.json()

{'danceability': 0.696,
 'energy': 0.905,
 'key': 2,
 'loudness': -2.743,
 'mode': 1,
 'speechiness': 0.103,
 'acousticness': 0.011,
 'instrumentalness': 0.000905,
 'liveness': 0.302,
 'valence': 0.625,
 'tempo': 114.944,
 'type': 'audio_features',
 'id': '11dFghVXANMlKmJXsNCbNl',
 'uri': 'spotify:track:11dFghVXANMlKmJXsNCbNl',
 'track_href': 'https://api.spotify.com/v1/tracks/11dFghVXANMlKmJXsNCbNl',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/11dFghVXANMlKmJXsNCbNl',
 'duration_ms': 207960,
 'time_signature': 4}

In [20]:
import time
import requests

def safe_request(url, headers, params, max_retries=20):
    retry_wait = 1
    for i in range(max_retries):
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            return response
        elif response.status_code == 429:
            retry_after = int(response.headers.get('Retry-After', retry_wait))
            print(f"Rate limit exceeded, retrying after {retry_after} seconds...")
            time.sleep(retry_after)
            retry_wait *= 2 
        else:
            response.raise_for_status()
    raise Exception("Max retries exceeded")

def search_tracks_by_genre(access_token, genre, limit=50, max_retries = 10):
    url = "https://api.spotify.com/v1/search"
    track_ids = []
    offset = 0

    while len(track_ids) < 500:
        query_params = {
            "q": f"genre:{genre}",
            "type": "track",
            "limit": limit,
            "offset": offset
        }
        headers = {"Authorization": f"Bearer {access_token}"}
        
        response = safe_request(url, headers=headers, params=query_params)
        data = response.json()
        tracks = data.get('tracks', {}).get('items', [])
        
        if not tracks:
            break

        for track in tracks:
            track_ids.append(track['id'])
        
        offset += limit
        if len(tracks) < limit:
            break

    return track_ids

genre = 'jazz'
track_ids = search_tracks_by_genre(access_token, genre)
print(len(track_ids), "tracks found for genre:", genre)
print(track_ids)


500 tracks found for genre: jazz
['1cyYZZeLCRhyNTZR8jXvsh', '6pf8OHiqUaWHDixLf3sSIz', '2lrPuMBd8bid1pNGRKnY7G', '1jfoM3mcBUZq3qug1Q6LSL', '0MzkbrMbHgXAo5fqh45wWx', '16mSw8BYiPdUn1psSkdHxY', '3ZeUjMpGFjbgp72Ns86oQg', '0hvUmpHzFe00kEb6AuTeJk', '23NRb46iByzITqHoluFH56', '2ilKEoYak67tNNp3Lshkxu', '02CofDgfgz4dS50xeEQQsE', '4MaQS1q7Jr7U6XxY3r9Zqb', '28debq2HT9lyLCo5bfM1cu', '1uuYYRNtGRXngyuzF6vsHx', '6WrUCuJCmI2jb3vcQ7ZRln', '22LJafGzlkYijQc4hZgi5S', '6bGS68FyN5SVz5I8bkXaNL', '2vXVFJkACuC12lxWW2DGLW', '3fa2skuFVh3QlLoMfYGW89', '1gLwIHn6qJfXvMQtHSp91W', '27hpxf0tACgaYvgnreG04l', '00F9YYircqhjooagEeDfL8', '7brl3DZJTZaUOZEGuAjJpH', '54bJGpx5WLCgEeFtRvoNcK', '03dOjHR0mMr6AK6Aw7zOvA', '4G3aqZQlg5knO1bfFB56lD', '7fFY8GR2xPv1YAvxVPwn6m', '3yGVWdaiq8K4o2wuiSPHZi', '1KXkusRNAWhbkqwdDjhbxY', '2hIqFq62MMXF8vrVRDh5HY', '5oS8QQGxUK1D58jPiAcqFW', '2TGo595fAxdx0Id5z3mmVc', '2sNyKHn7mHtWKKHuURl30Y', '0tAC0qTjy7muEbqbcDuDKX', '66SJLLbVcqer5VSBoT7Zqc', '4NjldrmE0aLQJ41wqbsn9w', '4s4Cz0E5VJJDCBF3qOmgZU', '2zj